In [1]:
from ultralytics import YOLO
import numpy as np
import cv2
import os
import pytesseract
import imutils
import easyocr
import re

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
reader = easyocr.Reader(['de'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [3]:
#trained yolo model
lp_detect_model = YOLO('yolo_model/best.pt')


In [4]:
#crop the license plate and save as image
def crop_license_plates(img, yolo_model, save_img=True):
    img_name = img.split("\\")[-1]
    
    
    print(f"Processing {img_name}")
    detected_lps = yolo_model.predict(img, verbose=False)[0]
    license_plates = []
    
    for lp in detected_lps.boxes.data.tolist():
        
        x1, y1, x2, y2, conf, _ = lp
       
        img_array = cv2.imread(img)
        cropped_lp =img_array[int(y1):int(y2), int(x1):int(x2), :]
        license_plates.append(cropped_lp)
        if save_img:
            cv2.imwrite(f"cropped_license_plates/cropped_{img_name}", cropped_lp)
        
        
    return license_plates

In [5]:
def ocr(image):
    custom_config = r'--psm 6 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZÖ0123456789'
    text = pytesseract.image_to_string(image, lang='deu', config=custom_config)
    if text:
        return text.strip("\n")
    return "not detected"

In [6]:
def get_new_w_h(rect):
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

    maxWidth = max(int(widthA), int(widthB))
    maxHeight = max(int(heightA), int(heightB))
    return maxWidth, maxHeight
    

In [7]:
def warp_image(img, contour):
    hull = cv2.convexHull(contour)

    # Get the minimum area rectangle from the convex hull
    rect = cv2.minAreaRect(hull)
    """
    approx = cv2.approxPolyDP(contour, 0.015 * cv2.arcLength(contour, True), True)
   
    
  
    print(approx.shape)
    points = approx.reshape(4, 2)
    rect = np.zeros((4, 2), dtype = "float32")
    s = points.sum(axis = 1)
    rect[0] = points[np.argmin(s)]
    rect[2] = points[np.argmax(s)]

    diff = np.diff(points, axis = 1)
    rect[1] = points[np.argmin(diff)]
    rect[3] = points[np.argmax(diff)]
    """
    maxWidth, maxHeight = get_new_w_h(rect)
    dst = np.array([[0, 0],
                    [maxWidth - 1, 0],
                    [maxWidth - 1, maxHeight - 1],
                    [0, maxHeight - 1]],
                    dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warp = cv2.warpPerspective(img.copy(), M, (maxWidth, maxHeight))
    return warp
    

In [8]:
def apply_image_processing(img, kernel):
    kernel = np.ones((3,3),np.uint8)
    # grayscale
    lp_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # erosion
    lp_gray = cv2.erode(lp_gray, kernel, iterations=1)
    # bilateral filter
    lp_gray = cv2.bilateralFilter(lp_gray, 5, 75, 75)
    # binary thresholding
    _, lp_threshold = cv2.threshold(lp_gray, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return lp_threshold

In [9]:
def preprocess_license_plate(license_plate, name, save_img=True):
    
    kernel = np.ones((3,3),np.uint8)
    lp_threshold = apply_image_processing(license_plate, kernel)
    
    contours, _ = cv2.findContours(lp_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    
    # contour with largest area
    max_contour = max(contours, key=cv2.contourArea)
    #warped_img = warp_image(license_plate.copy(), max_contour)
    
    # Crop the license plate region
    x, y, w, h = cv2.boundingRect(max_contour)
    license_plate_cropped = lp_threshold[y:y + h, x:x + w]

    if save_img:
        cv2.imwrite(f'thresholded_license_plates/thresh_bil_rectified_{name}', license_plate_cropped)

    return license_plate_cropped

In [10]:
#read characters on licenese plate using OCR  
def read_license_plates(license_plates,name, save_img=True):
    kernel = np.ones((3,3),np.uint8)
    for lp in license_plates:

        #pre processing of the image:
        
        #1.    grayscale the image
       
        lp_gray = cv2.cvtColor(lp, cv2.COLOR_BGR2GRAY)

        #2.     erode the image
        lp_gray = cv2.erode(lp_gray, kernel, iterations=1)

        #3a.     remove noise with median filter
        lp_median = cv2.medianBlur(lp_gray,5)
        
        #3b.     remove noise with bilateral filter
        lp_bilateral = cv2.bilateralFilter(lp_gray, 5, 75, 75)

        #4a.    canny edge detection 
        lp_canny = cv2.Canny(lp_median, 100, 200)

        #4b.     binary thresholding 
        _, lp_threshold = cv2.threshold(lp_median, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        _, lp_threshold2 = cv2.threshold(lp_bilateral, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        if save_img:
            cv2.imwrite(f'thresholded_license_plates/thresh_median_{name}', lp_threshold)
            cv2.imwrite(f'thresholded_license_plates/thresh_bil_{name}', lp_threshold2)
            cv2.imwrite(f'thresholded_license_plates/canny_{name}', lp_canny)

        return lp_threshold, lp_threshold2, lp_canny

In [11]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [12]:
import math
def compute_skew(src_img):

    if len(src_img.shape) == 3:
        h, w, _ = src_img.shape
    elif len(src_img.shape) == 2:
        h, w = src_img.shape
    else:
        print('upsupported image type')

    img = cv2.medianBlur(src_img, 3)

    edges = cv2.Canny(img,  threshold1 = 30,  threshold2 = 100, apertureSize = 3, L2gradient = True)
    lines = cv2.HoughLinesP(edges, 1, math.pi/180, 30, minLineLength=w / 4.0, maxLineGap=h/4.0)
    angle = 0.0
    nlines = lines.size

    #print(nlines)
    cnt = 0
    for x1, y1, x2, y2 in lines[0]:
        ang = np.arctan2(y2 - y1, x2 - x1)
        #print(ang)
        if math.fabs(ang) <= 30: # excluding extreme rotations
            angle += ang
            cnt += 1

    if cnt == 0:
        return 0.0
    return (angle / cnt)*180/math.pi

In [18]:
def pls_work(license_plate, name):
    img_g = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)
    img = cv2.bilateralFilter(img_g, 11, 17, 17)
    img = cv2.threshold(img, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    keypoints = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    max_c = max(contours, key=cv2.contourArea)
    max_c = cv2.approxPolyDP(max_c, 20, True)
    mask = np.zeros(img_g.shape, np.uint8)
    new_image = cv2.drawContours(mask, [max_c], 0,255, -1)

    new_image = np.where(mask == 255, img_g, 0)
    #rotated_img = warp_image(img, max_c)
    #TODO Sunday: maybe use minarearectangle to descew
    masked_img_gray = cv2.bilateralFilter(new_image, 3,32,32)
    masked_img_gray = cv2.threshold(masked_img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    #masked_img_gray = cv2.Canny(masked_img_gray, 30, 200)
    cv2.imwrite("test_results/masked_"+name, new_image)
    cv2.imwrite("test_results/processed_"+name, masked_img_gray)

    return img

folder = "test images 5.8/SHGLF206/sharp"
pattern = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789ÜÖ'
for img in os.listdir(folder):
    if img.endswith('.JPG') or img.endswith('.jpg'):
        img_path = os.path.join(folder, img)
        print(img_path)
        detected_licenseplates = crop_license_plates(img_path, lp_detect_model)
        if not detected_licenseplates:
            print( "No license plate detected!")
        else:
            test = pls_work(detected_licenseplates[0],img)

            res = reader.readtext(test, allowlist= pattern,detail=0,paragraph=True)

            print(res)
        #print(ocr(test))

test images 5.8/TÜAR2499/sharp\image_1.jpg
Processing image_1.jpg


FileNotFoundError: Image Not Found e:\Users\belau\Documents\BV-ML-CV-Praktikum\test images 5.8\TÜAR2499\sharp\image_1.jpg

In [14]:
def find_license_plate_id(img_file):
    print(img_file)
    detected_licenseplates = crop_license_plates(img_file, lp_detect_model)
    assert len(detected_licenseplates) == 1, "WARNING! More than one License plate detected. Access denied!"
    re_cropped = preprocess_license_plate(detected_licenseplates[0], img_file, save_img=False)
    test = pls_work(detected_licenseplates[0])
    
    res = reader.readtext(test, detail=0, paragraph=True)
    print(res)
    print(ocr(test))
    if re_cropped is not None:
        return ocr(re_cropped)
    return None

In [15]:
def main():
    path_to_cam_frames = 'PythonScripts/cam_frames'
    all_lp_ids = []
    img_files = [os.path.join(path_to_cam_frames, f) for f in os.listdir(path_to_cam_frames)
                 if f.endswith('.jpg') or f.endswith('.png') or f.endswith('.JPG')]
    all_lp_ids = map(find_license_plate_id, img_files)
    all_lp_ids = [lp_id for lp_id in all_lp_ids if lp_id is not None]
    print(all_lp_ids)
    predicted_license_plate_id = max(set(all_lp_ids), key=all_lp_ids.count)
    print(predicted_license_plate_id)
#main()   